<a href="https://colab.research.google.com/github/candeliat/RastgeleSayiUreteci/blob/main/RastgeleSayiUreteci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class LFSR_RNG:
    def __init__(self, tohum, tap_noktalari):
        """
        LFSR Başlatıcı
        tohum: Başlangıç değeri (Seed)
        tap_noktalari: XOR işlemine girecek bitlerin konumları
        """
        self.state = tohum
        self.taps = tap_noktalari

        if self.state == 0:
            self.state = 0b10101010

    def _bit_uret(self):
        yeni_bit = 0
        for tap in self.taps:
            bit_degeri = (self.state >> tap) & 1
            yeni_bit = yeni_bit ^ bit_degeri

        self.state = self.state >> 1

        if yeni_bit == 1:
            # 16-bitlik bir register varsayıyoruz
            self.state = self.state | (1 << 15)

        return yeni_bit

    def rastgele_sayi_uret(self, bit_uzunlugu=8):
        sonuc = 0
        for _ in range(bit_uzunlugu):
            uretilen_bit = self._bit_uret()
            sonuc = (sonuc << 1) | uretilen_bit
        return sonuc

# --- YENİ EKLENEN KISIM: SAYIYI METNE ÇEVİRME ---

def rastgele_metin_uret(lfsr_motoru, karakter_sayisi):
    """
    LFSR kullanarak belirtilen uzunlukta rastgele bir metin (string) üretir.
    """
    # 1. Kullanılabilecek karakter havuzu (Alfabemiz)
    # Harfler (Büyük/Küçük), Rakamlar ve Özel Karakterler
    harfler = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
    rakamlar = "0123456789"
    semboller = "!@#$%^&*"

    karakter_seti = harfler + rakamlar + semboller
    set_uzunlugu = len(karakter_seti)

    uretilen_metin = ""

    print(f"--- Metin Üretimi Başlıyor ({karakter_sayisi} karakter) ---")

    for i in range(karakter_sayisi):
        # 0-255 arası ham bir sayı alıyoruz
        ham_sayi = lfsr_motoru.rastgele_sayi_uret(bit_uzunlugu=8)

        # Modulo (%) işlemi ile sayıyı karakter setimizin uzunluğuna indirgiyoruz.
        # Örn: Sayı 200, Set Uzunluğu 70 ise -> 200 % 70 = 60. indeks
        secilen_indeks = ham_sayi % set_uzunlugu

        secilen_karakter = karakter_seti[secilen_indeks]
        uretilen_metin += secilen_karakter

        # Detay görmek istersen (Rapor için):
        # print(f"Adım {i+1}: Ham Sayı={ham_sayi} -> İndeks={secilen_indeks} -> Karakter='{secilen_karakter}'")

    return uretilen_metin

# --- TEST KISMI ---

# 1. Motoru Başlat
# Farklı 'tohum' değerleri farklı metinler üretir.
my_lfsr = LFSR_RNG(tohum=123456789, tap_noktalari=[0, 2, 3, 5])

# 2. Rastgele Parola / Metin Üret
parola = rastgele_metin_uret(my_lfsr, karakter_sayisi=12)
session_id = rastgele_metin_uret(my_lfsr, karakter_sayisi=32)

print("\n--- SONUÇLAR ---")
print(f"Üretilen Parola    : {parola}")
print(f"Üretilen SessionID : {session_id}")

# 3. Aynı tohumla tekrar denersek aynısı çıkacak mı? (Deterministik Test)
my_lfsr_reset = LFSR_RNG(tohum=123456789, tap_noktalari=[0, 2, 3, 5])
tekrar_parola = rastgele_metin_uret(my_lfsr_reset, karakter_sayisi=12)

print(f"Tekrar Üretim      : {tekrar_parola} (Aynısı olmalı)")

--- Metin Üretimi Başlıyor (12 karakter) ---
--- Metin Üretimi Başlıyor (32 karakter) ---

--- SONUÇLAR ---
Üretilen Parola    : TNUl44vaqDt@
Üretilen SessionID : LQz0A@m%qRXLaygmTkqPnYCIIcNmU@8H
--- Metin Üretimi Başlıyor (12 karakter) ---
Tekrar Üretim      : TNUl44vaqDt@ (Aynısı olmalı)
